In [16]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF
import pyspark.sql.functions as psf
from pyspark.sql.types import DoubleType

In [17]:
spark = SparkSession.builder.appName('TF-IDF').getOrCreate()
sc = spark.sparkContext

In [18]:
def split_file(x):
    value=x.split('\t')
    return (value[0], value[1].split(' '))

In [19]:
data = (sc.textFile('cleaned.txt')
    .map(lambda x: split_file(x))
    .toDF(['id', 'words'])
)

In [20]:
hashingTF = HashingTF(inputCol="words", outputCol='features', numFeatures=100000)
tf = hashingTF.transform(data)

In [21]:
idf = IDF(inputCol='features', outputCol='idf')
model = idf.fit(tf)
tf_idf = model.transform(tf)

In [22]:
cosine_similarity_udf = psf.udf(lambda x,y: round(float(x.dot(y)/(x.norm(2)*y.norm(2))), 2), DoubleType())

res = (tf_idf.alias("i").join(tf_idf.alias("j"), psf.col("i.id") < psf.col("j.id"))
    .select(
        psf.col("i.id").alias("i"), 
        psf.col("j.id").alias("j"), 
        cosine_similarity_udf("i.idf", "j.idf").alias("cosine_similarity"))
    .sort("i", "j"))
res.take(20)